# **Instituto de Informática - UFG**

Disciplina: Inteligência Artificial

Prof. Cedric Luiz de Carvalho

Grupo ( A )

---


Aluna(o): Luana Guedes Barros Martins

---

Aluna(o): Ruan Chaves Rodrigues

---

Aluna(o): Thiago Monteles de Souza

---

# Atividade Prática 06 - Processamento de Linguagem Natural

Particionamento das atividades:
- Luana Guedes Barros Martins: Desenvolvedor
- Ruan Chaves Rodrigues: Analista de dados
- Thiago Monteles de Souza: Gerente do Projeto

## Preparação do Ambiente Virtual

In [1]:
!python -m spacy download pt_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [2]:
!pip install nltk

In [12]:
!pip install gensim

In [14]:
# Imports

from collections import defaultdict
import numpy as np
from collections import Counter

import pandas as pd

import nltk
import spacy 
from gensim import corpora, models, similarities
from gensim.utils import simple_preprocess

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)
import seaborn as sns

## Descrição do Problema

Utilizando as técnicas estudadas, será desenvolvido uma ferramenta que permita a comparação de pequenos textos. Devem ser definidas métricas de comparação que classifique os textos segundo critérios de similaridade.

A ferramenta deverá ser aplicada aos Estudos Dirigidos desenvolvidos ao longo do curso de Inteligência Artificial 2019-3. Para cada estudo, deve-se construir uma matriz que mostre a similaridade entre os documentos. Depois, definir um documento de referência e traçar um gráfico de similaridade dos demais em relação a ele.

## Tratamento dos Dados

*Breve descrição do que foi feito*

In [15]:
"""
# Carregar o dataset "baseDeDados.csv" para o ambiente virtual

from google.colab import files
uploaded = files.upload()
"""

'\n# Carregar o dataset "baseDeDados.csv" para o ambiente virtual\n\nfrom google.colab import files\nuploaded = files.upload()\n'

In [16]:
docs = pd.read_csv("Dados - ED05.csv", sep=";")
docs.head()

,Resposta 1,Resposta 2,Resposta 3,Resposta 4,Resposta 5,Resposta 6,Resposta 7,Resposta 8,Resposta 9,Resposta 10,...,Resposta 32,Resposta 33,Resposta 34,Resposta 35,Resposta 36,Resposta 37,Resposta 38,Resposta 39,Resposta 40,Resposta 41
0,O desenvolvimento de redes neurais artificiais...,SÃO UM MODELO MATEMÁTICO DE NEURÔNIO ARTIFICIA...,"OS PRINCIPAIS COMPONENTES SÃO: DENDRITOS, CORP...",São as conexões que interligam as camadas de n...,As redes de neurônios biológicas trabalham de ...,São sistemas computacionais distribuídos compo...,A arquitetura de uma rede neural artificial es...,O aprendizado diz respeito às regras utilizada...,O funcionamento é o seguinte: \n\n 1 - Cada te...,As funções de ativação permite a saída de um ...,...,Pois os valores de erro são conhecidos apenas ...,1º Valores aleatórios de pesos são inicializad...,"Utilizando o conceito de momentum, que quanti...","Utilizado o conceito de momentum, que quantifi...",Para reduzir a ocorrência de overfitting. O co...,Quando as taxas de erro de validação de uma re...,A lentidão que a mesma leva para convergir com...,Os seguintes fatores:\n\n\t*\n\nNúmero de exe...,Por força bruta. Cada tipo de arquitetura é te...,"Empírica: Por busca cega, arquiteturas são tes..."
1,O fundamento natural das redes neurais artific...,Os neurônios artificiais são conhecidos como u...,Os principais componente de um neurônio são: d...,As sinapses são as unidades que medeiam as int...,Os neurônios biológicos possuem um tempo de ex...,As redes neurais artificiais são baseadas em m...,A arquitetura de uma RNA está relacionada ao t...,O aprendizado de uma RNA diz respeito as regra...,O neurônio é a unidade de processamento fundam...,As funções de ativação servem para definir a s...,...,Como os valores dos erros são conhecidos apena...,-,O valor da taxa de aprendizado n tem uma forte...,Uma possível medida para amenizar esse problem...,Para reduzir a ocorrência de overfitting a par...,Plotando um grafico com as taxas de erros dos ...,Uma crítica feita ao algoritmo back-propagatio...,O número de neurônios de uma camada intermediá...,É preciso levar em consideração qual função de...,• Empírica: consiste na realização de uma busc...
2,As redes neurais artificiais (RNAs) tomou como...,É O MODELO MATEMÁTICA DE NEURÔNIO ARTIFICIAL P...,A unidade fundamental do sistema nervoso é o n...,O CONTATO ENTRE A TERMINAÇÃO DE UM AXÔNIO E O ...,APESAR DE OS NEURÔNIOS BIOLÓGICOS POSSUÍREM UM...,AS RNAS SÃO SISTEMAS COMPUTACIONAIS DISTRIBUÍD...,A ARQUITETURA ESTÁ RELACIONADA AO TIPO E NÚMER...,O APRENDIZADO DIZ RESPEITO ÀS REGRAS UTILIZADA...,"CADA TERMINAL DE ENTRADA DO NEURÔNIO, SIMULAND...",AS FUNÇÕES DE ATIVAÇÃO SERVEM PARA DEFINIR A S...,...,POR QUE APENAS OS VALORES DOS ERROS DA CAMADA ...,A REDE É INICIALIZADO COM VALORES ALEATÓRIOS P...,O VALOR DA TAXA DE APRENDIZADO TEM UMA FORTE I...,O MOMENTUM É UMA MEDIDA QUE QUANTIFICA A IMPOR...,SE SEPARA OS DADOS EM CONJUNTO DE TREINO E DE ...,"EM UM DADO MOMENTO, SE A TAXA DE ERRO DE VALID...",UMA CRÍTICA FEITA É A SUA LENTIDÃO NA CONVERGÊ...,Os fatores que determina o número de neurônios...,Empírica: consiste na realização de uma busca ...,Empírica: consiste na realização de uma busca ...
3,O FUNDAMENTO NATURAL DAS REDES NEURAIS ARTIFIC...,OS NEURÔNIOS ARTIFICIAIS SÃO CONHECIDOS COMO U...,OS PRINCIPAIS COMPONENTE DE UM NEURÔNIO SÃO: D...,AS SINAPSES SÃO AS UNIDADES QUE MEDEIAM AS INT...,OS NEURÔNIOS BIOLÓGICOS POSSUEM UM TEMPO DE EX...,AS REDES NEURAIS ARTIFICIAIS SÃO BASEADAS EM M...,A ARQUITETURA DE UMA RNA ESTÁ RELACIONADA AO T...,O APRENDIZADO DE UMA RNA DIZ RESPEITO ÀS REGRA...,O neurônio é a unidade de processamento fundam...,AS FUNÇÕES DE ATIVAÇÃO SERVEM PARA DEFINIR A S...,...,COMO OS VALORES DOS ERROS SÃO CONHECIDOS APENA...,RECEBE COMO ENTRADA UM CONJUNTO N DE OBJETOS D...,O VALOR DA TAXA DE APRENDIZADO N TEM UMA FORTE...,UMA POSSÍVEL MEDIDA PARA AMENIZAR ESSE PROBLEM...,PARA REDUZIR A OCORRÊNCIA DE OVERFITTING A PAR...,PLOTANDO UM GRÁFICO COM AS TA

In [17]:
docs = docs.values.flatten().tolist()

In [18]:
ids = pd.read_csv("Dados - ED05.csv", sep=";").values

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/luana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
stopwords = nltk.corpus.stopwords.words('portuguese')
nlp = spacy.load("pt_core_news_sm")

In [21]:
ids = [ [(idx2, idx) for idx,v in enumerate(v2) ] for idx2,v2 in enumerate(ids) ]
ids = [x for y in ids for x in y ]

In [22]:
docs = [nlp(x) for x in docs]
docs = [ [ y for y in x if y.is_punct == False and y.text.lower() not in stopwords] for x in docs ]

TypeError: object of type 'float' has no len()

In [ ]:
tokens = [ [ y.lemma_ for y in x ] for x in docs ]

In [ ]:
counter = dict(Counter([ x for y in tokens for x in y ]))
low_freq = [x for x in counter.keys() if counter[x] == 1]
tokens = [ x for x in tokens if x not in low_freq ]

In [ ]:
token_docs = [ ' '.join(x) for x in tokens ]
mydict = corpora.Dictionary([simple_preprocess(line) for line in token_docs])
nf = len(mydict.dfs)

In [ ]:
corpus = [mydict.doc2bow(simple_preprocess(line)) for line in token_docs]

In [ ]:
for doc in corpus:
    print([[mydict[id], freq] for id, freq in doc])

## Definição do Modelo

In [ ]:
tfidf = models.TfidfModel(corpus)

In [ ]:
for doc in tfidf[corpus]:
    print([[mydict[id], np.around(freq, decimals=2)] for id, freq in doc])

In [ ]:
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=nf)

## Resultado

In [ ]:
dataplot = []
for item in corpus:
    dataplot.append(index[tfidf[item]])

In [ ]:

sns.heatmap(pd.DataFrame(dataplot))

In [ ]:
df = pd.DataFrame(dataplot)

In [ ]:
tmp = df.applymap(lambda x: 'nan' if x < 0.9 else x)
arr = tmp.values.tolist()
matches = []
for idx, item in enumerate(arr):
    for idx2, item2 in enumerate(item):
        if idx != idx2 and arr[idx][idx2] != 'nan':
            matches.append([idx, idx2])

In [ ]:
pairs = [ [ ids[x][0] for x in y ] for y in matches ]
pairs[0:10]

In [ ]:
dict_pairs = dict(Counter([ tuple(x) for x in pairs ])).items()

dict_pairs

In [ ]:
sns.heatmap(dataplot)